# Практическая работа к теме «1.5. Обработка данных в Python. Библиотека Pandas.»

## 1. Импорт библиотек

In [39]:
import json
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## 2. Подготовка датасета

В видео к практической работе был представлен более сложный и времязатратный способ формирования датафрейма. Я не стал множить сущности и воспользовался вариантом ниже, т.к. формировать датафреймы вручную и писать циклы умею. Если Вы сочтете, что я "читер", то могу обработать json циклами и сформировать таблицу также как это было показано в видео :)

In [ ]:
# Открываем json
with open('parsed.json', 'r', encoding='utf8') as file:
    data = json.load(file)

In [ ]:
# Превращаем его в DataFrame
df = pd.DataFrame(data['data'])
df

In [ ]:
# Подкорректируем названия колонок
df.columns = ['title', 'work_experience', 'salary', 'region']

In [ ]:
# Запишем датасет в csv-таблицу
df.to_csv('parsed_hh.csv', index=False)

## 3. Исследовательский анализ, подготовка и предобработка данных

In [40]:
# Откроем наш датасет из уже записанного csv-файла
df = pd.read_csv('parsed_hh.csv')
df

,title,work_experience,salary,region
0,Python Разработчик,1–3 года,от 200 000 до 1 500 000 KZT на руки,Астана
1,Full-stack Python Developer,3–6 лет,от 4 500 до 5 000 EUR до вычета налогов,Астана
2,Junior разработчик/программист в Python,не требуется,от 150 000 до 200 000 KZT до вычета налогов,Нур-Султан (Астана)
3,Python Developer (Middle/Senior),3–6 лет,от 2 000 до 4 000 USD на руки,Алматы
4,Backend разработчик,1–3 года,з/п не указана,Астана
...,...,...,...,...
4279,Специалист службы поддержки с техническими зна...,не требуется,от 15 000 до 40 000 руб. на руки,Ижевск
4280,Web-разработчик junior,не требуется,до 70 000 руб. на руки,Санкт-Петербург
4281,Web-разработчик,не требуется,з/п не указана,Нижний Новгород
4282,Младший аналитик (IT),не требуется,от 100 000 до 130 000 руб. на руки,Москва


In [41]:
# Выведем общую информацию о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4284 entries, 0 to 4283
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            4283 non-null   object
 1   work_experience  4284 non-null   object
 2   salary           4284 non-null   object
 3   region           4284 non-null   object
dtypes: object(4)
memory usage: 67.0+ KB


Имеем 4 столбца и 4284 строки. Есть один пропуск в столбце 'title'.  
Сделам проверку на полные дубликаты данных:

In [42]:
df.duplicated().sum()

0

Дубликаты отсутствуют.

Далее значения из столбцов `salary` и `work_experience` переведем в числовой формат.  
В отдельный столбец выведем значение валюты.  
Так как многие зарплаты и опыт указаны в виде диапазона, то добавим дополнительные стобцы в датасет (пока с пустыми ячейками), в которые в дальнейшем запишем минимальное значение из диапазона и максимальное значение.

In [43]:
# Добавляем новые столбцы
df['salary_min'] = np.nan
df['salary_max'] = np.nan
df['exp_min'] = np.nan
df['exp_max'] = np.nan

In [44]:
df.insert(3, 'currency', 0)

Для того, чтобы выделить числовые значения из текста и определить валюту, в которой указана зарплата напишем две функции для одной строки, которые позже применим к датасету методом `apply()`. 

- `salary_to_num` - переведет данные о зарплате в числа и и определит валюту;
- `exp_to_num` - переведет данные о требуемом опыте в числа.

In [45]:
def salary_to_num(row):
    number_1 = []
    number_2 = []
    last_symbol = 'start'
    m = ''
    val = row['salary']
    salary = str(row['salary']).replace(' ', '')
    salary = list(''.join(salary))
    salary_min = row['salary_min']
    salary_max = row['salary_max']
    currency = ''
    
    for i in salary:
        if m.isnumeric() == True and i.isnumeric() == False:
            last_symbol = 'next'
        if last_symbol == 'start':
            if i.isnumeric() == True:
                number_1.append(i)
        elif last_symbol == 'next':
            if i.isnumeric() == True:
                number_2.append(i)
        m = i

    if 'EUR' in str(val):
        currency = 'EUR'
    if 'USD' in str(val):
        currency = 'USD'
    if 'KGS' in str(val): 
        currency = 'KGS'
    if 'KZT' in str(val): 
        currency = 'KZT'
    if ' сум ' in str(val): 
        currency = 'UZS'
    if 'руб.' in str(val): 
        currency = 'RUB'
    if 'бел. руб.' in str(val): 
        currency = 'BYN'
    if 'з/п не указана' in str(val):
        currency = 'missing'
    
    try:
        number_1 = int(''.join(number_1))
    except:
        number_1 = 0
    try:
        number_2 = int(''.join(number_2))
    except:
        number_2 = 0
    
    if number_1 != 0 and number_2 != 0:
        row.loc['salary_min'] = number_1
        row.loc['salary_max'] = number_2
    elif number_1 != 0 and number_2 == 0:
        row.loc['salary_min'] = 0
        row.loc['salary_max'] = number_1
    else:
        row.loc['salary_min'] = 0
        row.loc['salary_max'] = 0
        
    row.loc['currency'] = currency
    row.loc['salary'] = val
    
    return row


In [46]:
def exp_to_num(row):
    number_1 = []
    number_2 = []
    last_symbol = 'start'
    m = ''
    val = row['work_experience']
    exp = str(row['work_experience']).replace(' ', '')
    exp = list(''.join(exp))
    exp_min = row['exp_min']
    exp_max = row['exp_max']
    
    for i in exp:
        if m.isnumeric() == True and i.isnumeric() == False:
            last_symbol = 'next'
        if last_symbol == 'start':
            if i.isnumeric() == True:
                number_1.append(i)
        elif last_symbol == 'next':
            if i.isnumeric() == True:
                number_2.append(i)
        m = i
    
    try:
        number_1 = int(''.join(number_1))
    except:
        number_1 = 0
    try:
        number_2 = int(''.join(number_2))
    except:
        number_2 = 0
    
    if number_1 != 0 and number_2 != 0:
        row.loc['exp_min'] = number_1
        row.loc['exp_max'] = number_2
    elif number_1 != 0 and number_2 == 0:
        row.loc['exp_min'] = number_1
        row.loc['exp_max'] = number_1
    else:
        row.loc['exp_min'] = 0
        row.loc['exp_max'] = 0
        
    row.loc['work_experience'] = val
    
    return row

In [47]:
# Применим функции к датасету
df = df.apply(salary_to_num, axis=1)
df = df.apply(exp_to_num, axis=1)

In [48]:
df

,title,work_experience,salary,currency,region,salary_min,salary_max,exp_min,exp_max
0,Python Разработчик,1–3 года,от 200 000 до 1 500 000 KZT на руки,KZT,Астана,200000,1500000,1,3
1,Full-stack Python Developer,3–6 лет,от 4 500 до 5 000 EUR до вычета налогов,EUR,Астана,4500,5000,3,6
2,Junior разработчик/программист в Python,не требуется,от 150 000 до 200 000 KZT до вычета налогов,KZT,Нур-Султан (Астана),150000,200000,0,0
3,Python Developer (Middle/Senior),3–6 лет,от 2 000 до 4 000 USD на руки,USD,Алматы,2000,4000,3,6
4,Backend разработчик,1–3 года,з/п не указана,missing,Астана,0,0,1,3
...,...,...,...,...,...,...,...,...,...
4279,Специалист службы поддержки с техническими зна...,не требуется,от 15 000 до 40 000 руб. на руки,RUB,Ижевск,15000,40000,0,0
4280,Web-разработчик junior,не требуется,до 70 000 руб. на руки,RUB,Санкт-Петербург,0,70000,0,0
4281,Web-разработчик,не требуется,з/п не указана,missing,Нижний Новгород,0,0,0,0
4282,Младший аналитик (IT),не требуется,от 100 000 до 130 000 руб. на руки,RUB,Москва,100000,130000,0,0


In [49]:
# Проверим какие виды валюты удалось выделить
df['currency'].value_counts()

missing    3036
RUB         972
USD         128
EUR          88
KZT          35
             19
UZS           3
BYN           2
KGS           1
Name: currency, dtype: int64

In [50]:
# Среди значений валюты есть пустое. 
# Выведем строки, которые ему соответствуют на экран.
df[df['currency']=='']

,title,work_experience,salary,currency,region,salary_min,salary_max,exp_min,exp_max
47,NaN,undefinite,undefinite,,undefinite,0,0,0,0
235,undefinite,undefinite,RocketData,,Минск,0,0,0,0
742,undefinite,undefinite,Ventra,,Москва,0,0,0,0
1333,undefinite,undefinite,Leads,,Москва,0,0,0,0
1467,Разработчик C# алгоритмыз/п не указана,1–3 года,ООО Гудфокаст,,Калужская,0,0,1,3
1694,undefinite,undefinite,ООО Защищенные Телекоммуникации,,Тула,0,0,0,0
1872,Team lead / Tech lead / Технический лидерз/п н...,3–6 лет,Convergent,,Москва,0,0,3,6
2040,Инженер-программистз/п не указана,1–3 года,МУП Водоканал,,Динамо,0,0,1,3
2195,IOS developerот 150 000 руб. на руки,1–3 года,ООО Волкрафт,,Penza,0,0,1,3
2285,Технический директор / Head of Techз/п не указана,3–6 лет,Convergent,,Москва,0,0,3,6


In [51]:
# В столбце с з/п некорректные данные. 
# Удалим эти строки из датасета
df.drop(index=df[df['currency']==''].index, inplace=True, axis=1)

In [52]:
# Заново проверим виды валют
df['currency'].value_counts()

missing    3036
RUB         972
USD         128
EUR          88
KZT          35
UZS           3
BYN           2
KGS           1
Name: currency, dtype: int64

Значения корректные.  
Проверим датасет на пропуски

In [53]:
df.isna().sum()

title              0
work_experience    0
salary             0
currency           0
region             0
salary_min         0
salary_max         0
exp_min            0
exp_max            0
dtype: int64

Пропусков больше нет.

In [54]:
# Сбросим индексацию датасета
df.reset_index(inplace=True, drop=True)

Проверим строки, в которых может встречаться значение 'undefinite'.

In [55]:
columns_list = df.columns
wrong_list = []
for column in columns_list:
    if len(df[df[column]=='undefinite']) > 0:
        for i in df[df[column]=='undefinite'].index:
            if i not in wrong_list:
                wrong_list.append(i)
wrong_list

[]

Такие строки отсутствуют.

In [56]:
# Размер датасета после очистки от неверных данных и добавления новой информации
df.shape

(4265, 9)

В датасете есть много данных о зарплате указанных в следующем виде: 'до 70 000 руб. на руки'.
При таком указании заработной платы нам известно только максимальное вознаграждение, а на месте минимального установлен 0, что является некорректным.
Попробуем сгладить эти неточности и восполнить нули.
Ноль можно было бы заменить на значение МРОТ, но у нас в регионах присутствуют города из разных государств, поэтому такой вариант не подходит.
Восполним нули в 2 шага.

**ШАГ 1:**
  - Отфильтровываем датасет по региону и записываем в переменную `filtered_region`;
  - В полученной таблице находим строки, в которых указана минимальная и максимальная зарплата и записываем их в переменную `filtered_salary`;
  - Также в датасете `filtered_region` ищем строки, в которых указаны только максимальная зарплата, а минимальная равна нулю;
  - Берем строку, где есть данные о максимальной зарплате, а минимальная нулевая. Далее фильтруем датасет по значению найд енной максимальной з/п. Полученный на выходе датасет используем для расчета медианной минимальной зарплаты.
  - Ноль заменяем на полученное значение медианы. Таким образом заполняем все возможные нули, а там где это не возможно возвращаем в ячейку прежнее значение. Проделываем эту операцию в цикле.

**ШАГ 2:**
  - К этому шагу переходим для восполнения оставшихся нулей. Так как у нас сильный невосполнимый дефицит данных, то есть два пути: либо мы удаляем эти строки (если процент таких данных от общего количества данных будет 1-3% и не более), либо мы можем воспользоваться методом Ферми и, например, заменить нули половиной суммы максимально указанной работодателем зарплаты. В любом случае искажения данных неизбежны, но эти действия помогут сделать их не столь существенными.
  


In [57]:
# Оценим количество строк, где нет минимальной з/п, но есть максимальная
wrong_row = df[(df['salary_min']==0)&(df['salary_max']!=0)].shape[0]
print(f'Количество строк = {wrong_row}, что составляет {wrong_row*100/len(df):.2f}% от объема данных всего датасета.')

Количество строк = 567, что составляет 13.29% от объема данных всего датасета.


В датасете имеются города, записанные с дополнительным словом 'город'. Пример ниже. Это может помешать фильтрации данных по региону.

In [58]:
df[df['region']=='город Лимасол'].head(3)

,title,work_experience,salary,currency,region,salary_min,salary_max,exp_min,exp_max
12,Python Developer (Crypto),3–6 лет,от 5 000 EUR на руки,EUR,город Лимасол,0,5000,3,6
91,"Senior Full-stack Developer (to Cyprus, Armenia)",3–6 лет,от 5 000 EUR на руки,EUR,город Лимасол,0,5000,3,6
131,"Senior QA Engineer (Automation, Python) to Cyp...",3–6 лет,от 4 500 EUR на руки,EUR,город Лимасол,0,4500,3,6


In [59]:
# Перестрахуемся и уберем приставку 'город' там, где это возможно
def city_remove(row):
    region = row['region']
    if 'город' in region:
        region = region.replace('город ', '')
        return region
    else:
        return region
        
df['region'] = df.apply(city_remove, axis=1)

Теперь можно перейти к шагу 1 в восполнении данных с нулями.

In [60]:
# Шаг 1
region_list = df['region'].unique()
for region in region_list:
    # отфильтровываем датасет по региону
    filtered_region = df[df['region'] == region]
    index_f_r = filtered_region.index
    # Из датасета, отфильтрованного по региону, отфильтровываем строки с мин. з/п != 0 и макс. з/п != 0
    filtered_salary = filtered_region[(filtered_region['salary_min'] != 0) &
                                    (filtered_region['salary_max'] != 0)]
    
    for i in index_f_r:
        similar = filtered_salary[filtered_salary['salary_max'] == filtered_region['salary_max'][i]]
        if (filtered_region['salary_min'][i] == 0 and filtered_region['salary_max'][i] != 0 
            and len(filtered_salary) != 1 and len(filtered_salary) != 0):
            if len(similar) > 1:
                s_median = similar['salary_min'].median()
                df['salary_min'][i] = s_median
            else:
                df['salary_min'][i] = df.loc[i, 'salary_min']
        elif (filtered_region['salary_min'][i] == 0 and filtered_region['salary_max'][i] != 0) and (len(filtered_salary) == 1 or len(filtered_salary) == 0):
            df['salary_min'][i] = df.loc[i, 'salary_min']

df['salary_min'] = df['salary_min'].astype('int')

In [61]:
# Оценим количество строк, где нет минимальной з/п, но есть максимальная
wrong_row = df[(df['salary_min']==0)&(df['salary_max']!=0)].shape[0]
print(f'Количество строк = {wrong_row}, что составляет {wrong_row*100/len(df):.2f}% от объема данных всего датасета.')

Количество строк = 334, что составляет 7.83% от объема данных всего датасета.


Удалось восполнить 233 строки. Переходим к шагу 2.

In [62]:
def del_zero(row):
    s_min = row['salary_min']
    s_max = row['salary_max']
    if s_min == 0 and s_max != 0:
        fixed_salary = int(s_max / 2)
        return fixed_salary
    else:
        return s_min

In [63]:
df['salary_min'] = df.apply(del_zero, axis=1)

In [64]:
df.head()

,title,work_experience,salary,currency,region,salary_min,salary_max,exp_min,exp_max
0,Python Разработчик,1–3 года,от 200 000 до 1 500 000 KZT на руки,KZT,Астана,200000,1500000,1,3
1,Full-stack Python Developer,3–6 лет,от 4 500 до 5 000 EUR до вычета налогов,EUR,Астана,4500,5000,3,6
2,Junior разработчик/программист в Python,не требуется,от 150 000 до 200 000 KZT до вычета налогов,KZT,Нур-Султан (Астана),150000,200000,0,0
3,Python Developer (Middle/Senior),3–6 лет,от 2 000 до 4 000 USD на руки,USD,Алматы,2000,4000,3,6
4,Backend разработчик,1–3 года,з/п не указана,missing,Астана,0,0,1,3


In [65]:
# Снова оценим количество строк, где нет минимальной з/п, но есть максимальная
wrong_row = df[(df['salary_min']==0)&(df['salary_max']!=0)].shape[0]
print(f'Количество строк = {wrong_row}, что составляет {wrong_row*100/len(df):.2f}% от объема данных всего датасета.')

Количество строк = 0, что составляет 0.00% от объема данных всего датасета.


Все возможные нули заменены. Дополнительно сбросим индексацию датасета.

In [66]:
df.reset_index(inplace=True, drop=True)

Так как почти все зарплаты и опыт работы указаны в виде диапазонов, то для более-менее корректной оценки введем два дополнительных столбца со средним значением из этих диапазонов. 

In [67]:
df['salary_mean'] = (df['salary_min'] + df['salary_max']) / 2
df['exp_mean'] = (df['exp_min'] + df['exp_max']) / 2
df['salary_mean'] = df['salary_mean'].apply(lambda x: int(x + 0.5))
df['exp_mean'] = df['exp_mean'].apply(lambda x: int(x + 0.5))

In [68]:
df.head()

,title,work_experience,salary,currency,region,salary_min,salary_max,exp_min,exp_max,salary_mean,exp_mean
0,Python Разработчик,1–3 года,от 200 000 до 1 500 000 KZT на руки,KZT,Астана,200000,1500000,1,3,850000,2
1,Full-stack Python Developer,3–6 лет,от 4 500 до 5 000 EUR до вычета налогов,EUR,Астана,4500,5000,3,6,4750,5
2,Junior разработчик/программист в Python,не требуется,от 150 000 до 200 000 KZT до вычета налогов,KZT,Нур-Султан (Астана),150000,200000,0,0,175000,0
3,Python Developer (Middle/Senior),3–6 лет,от 2 000 до 4 000 USD на руки,USD,Алматы,2000,4000,3,6,3000,5
4,Backend разработчик,1–3 года,з/п не указана,missing,Астана,0,0,1,3,0,2


## 4. Группировка по типам вакансий

Добавим столбец `vacancy_type`, в который позже запишем типы вакансий.  
Группировку вакансий будем делать по этим типам.  
Разделение по тимам будет приблизительным, так как среди наименований вакансий есть те, которые могут относиться сразу к нескольким категориям и типам. Также названия вакансий составлены из русских и английских слов, поэтому провести лемматизацию слов автоматизировано затруднительно.

In [69]:
# Добавляем столбец
df.insert(1, 'vacancy_type', 0)

Разделим вакансии на следующие группы:

№ п/п | Направление | Код
---|---|---
1 | Data Science | DS
2 | Искусственный интеллект | AI
3 | Инженер данных | DE
4 | Большие данные | BD
5 | Глубокое обучение | DL
6 | Анализ данных | DA
7 | Машинное обучение | ML
8 | Software Engenering | SE
9 | Web-разработка | WB
9 | Прочее | other




Составим списки ключевых слов для поиска вакансий.

In [70]:
keywords_dict = {
    'ds_list': ['data science', 'data-science', 'data scientist', 'data-scientist'],

    'ai_list': [' ии',' ии ', 'ai ', ' ai ', 
                'искусственный интеллект', 'искусственного интеллекта', 'artificial intelligence'],

    'de_list': ['data engeener', 'data engeenering', 'инженер данных', 'инженер больших данных', 
                'дата-инженер', 'data-engeener'],

    'bd_list': ['big data', 'big-data', 'большие данные', 'больших данных'],

    'dl_list': [' dl', ' dl ', 'dl ', 'dl-', 
                'deep learning', 'deep-learning', 'глубокое обучение', 'глубокому обучению', 
                'computer vision'],

    'da_list': ['data analysis', 'data-analysis', 'data analytics', 
                'data-analytics', 'data analyst', 'data-analyst', 
                'аналитик данных', 'data аналитик', 'data аналитики', 
                'data аналитика', 'data-аналитик', 'data-аналитики', 
                'data-аналитика', 'анализ данных', 'анализу данных', 
                'аналитики данных', 'аналитика данных', 'data анализ', 
                'data анализа', 'data анализу', 'data-анализ', 
                'data-анализа', 'data-анализу', 'аналитик', 
                'аналитики', 'аналитику'],

    'ml_list': ['machine learning', 'machine-learning', 'машинное обучение', 'машинного обучения'],

    'se_list': ['software engeener', 'software engeenering', 'software-engeener',
                'software-engeenering', 'software developer', 'software developing',
                'software develop', 'software-developer', 'software-developing',
                'software-develop', 'разработчик', 'разработчика', 
                'разработка', 'программного обеспечения', 'программное обеспечение', 
                'software инженер', 'software-инженер', 'инженер', 
                'developer'],
    
    'wb_list': ['web', ' web', 'web ', ' web ',
                'web-разработчик', 'web разработчик', 'web-разработка', 'web разработка',
                'web-разработки', 'web разработки','web-разработку', 'web разработку',
                'верстка', 'верстальщик', 'php', 'php-разработчик', 
                'javascript', 'javascript-разработчик', 'javascript разработчик', 'лендинг', 
                'сайт', 'сайтов', 'laravel', 'django', 
                'веб-разработчик', 'веб разработчик', 'pyramid', 'web2py', 
                'веб-разработка', 'веб разработка', 'веб-разработку', 'веб разработку',
                'веб-разработки', 'веб разработки', 'flask', 'bottle',  
                'tornado', 'cherrypy', 'codeigniter']}

In [71]:
# Посчитаем количество уникальных вакансий
title_unique = df['title'].unique()
title_unique.shape

(2909,)

In [72]:
def typing(typing_dict, column, code_column, data):
    for key in typing_dict:
        category = key.replace('_list', '').upper()
        for v in typing_dict[key]:
            for i in range(len(data)):
                if (v in str(column[i]).lower()) and (code_column[i] == 0):
                    code_column[i] = category
    for n in range(len(data)):
        if code_column[n] == 0:
            code_column[n] = 'other'

In [73]:
typing(keywords_dict, df['title'], df['vacancy_type'], df)

In [74]:
df['vacancy_type'].value_counts()

SE       2372
other    1472
DA        247
DS         61
WB         51
BD         29
ML         12
DL          9
AI          7
DE          5
Name: vacancy_type, dtype: int64

Категоризация вакансий проведена успешно.

## 5. Дополнительные расчеты и анализ данных

### 5.1. Средняя и медианная зарплата

#### 5.1.1. Расчет по регионам

##### 5.1.1.1. Категоризация по странам

Удобным будет выполнить категоризацию по странам.
Это можно было сделать изначально при парсинге вакансий, но в задании не стояло задачи указывать именно страну. Поэтому в настоящий момент имеем в датасете набор городов из разных государств.  
Для того, чтобы подобрать к городу страну, в которой этот город находится установим и импортируем несколько дополнительных модулей: 
 - `geopy` - чтобы получить информацию о местоположении;
 - `googletrans` - для перевода иностранного названия страны на русский язык.


In [75]:
#pip install geopy
from geopy.geocoders import Nominatim
#pip install googletrans==3.1.0a0
import googletrans
from googletrans import Translator

In [76]:
# Создаем объекты словаря и геолокатора
geolocator = Nominatim(user_agent = "geoapiExercises")
translator = Translator() 

In [77]:
# Напишем функцию для подбора страны по городу
def city_to_country(row):
    city = row['region']
    location = geolocator.geocode(str(city))
    location = str(location).split()[-1]
    result = translator.translate(location, dest='ru')

    return result.text

Перед началом подбора стран проверим исходные данные в столбце `region`.

In [78]:
list(df['region'].unique())

['Астана',
 'Нур-Султан (Астана)',
 'Алматы',
 'Семей',
 'Лимасол',
 'Атырау',
 'Костанай',
 'Уральск',
 'Аксай (Казахстан)',
 'Караганда',
 'Щучинск',
 'Лимассол',
 'Ljubljana',
 'Павлодар',
 'Баку',
 'Гянджа',
 'Минск',
 'Могилев',
 'Гродно',
 'Витебск',
 'Минский район',
 'повят Варшава',
 'Гомель',
 'Брест',
 'Тбилиси',
 'Батуми',
 'Каспи',
 'Боржоми',
 'Кутаиси',
 'Абхазия',
 'Ростов-на-Дону',
 'София',
 'Армения',
 'Латвия',
 'Сербия',
 'Кишинёв',
 'Турция',
 'Сиэтл',
 'Кипр',
 'Финляндия',
 'Черногория',
 'Москва',
 'Ереван',
 'Литва',
 'ОАЭ',
 'Венгрия',
 'Румыния',
 'Болгария',
 'округ Белград',
 'Германия',
 'Белград',
 'Польша',
 'Вьетнам',
 'Берлин',
 'Канада',
 'Бангладеш',
 'Бишкек',
 'Ташкент',
 'Фергана',
 'Навои',
 'Акташ (Узбекистан)',
 'Екатеринбург',
 'эмират Дубай',
 'Новосибирск',
 'Тюмень',
 'Санкт-Петербург',
 'Калининград',
 'Казань',
 'Тула',
 'Иркутск',
 'Симферополь',
 'Самара',
 'Владимир',
 'Новокузнецк',
 'Томск',
 'Севастополь',
 'Магнитогорск',
 'Омск',

Исправим данные, которые неверно записаны и могут помешать корректному определению страны.  
Данные, где указаны страны, вместо города заменим на наименования столиц этих стран. Некорректные данные запишем в корректном виде.

In [79]:
df.loc[df['region'] == 'City of Limassol', 'region'] = 'Лимасол'
df.loc[df['region'] == 'Аксай (Казахстан)', 'region'] = 'Аксай'
df.loc[df['region'] == 'Ljubljana', 'region'] = 'Любляна'
df.loc[df['region'] == 'Минский район', 'region'] = 'Минск'
df.loc[df['region'] == 'повят Варшава', 'region'] = 'Варшава'
df.loc[df['region'] == 'ОАЭ', 'region'] = 'Дубай'
df.loc[df['region'] == 'округ Белград', 'region'] = 'Белград'
df.loc[df['region'] == 'Вьетнам', 'region'] = 'Ханой'
df.loc[df['region'] == 'Германия', 'region'] = 'Берлин'
df.loc[df['region'] == 'Финляндия', 'region'] = 'Хельсинки'
df.loc[df['region'] == 'Абхазия', 'region'] = 'Сухум'
df.loc[df['region'] == 'Армения', 'region'] = 'Ереван'
df.loc[df['region'] == 'Латвия', 'region'] = 'Рига'
df.loc[df['region'] == 'Сербия', 'region'] = 'Белград'
df.loc[df['region'] == 'Турция', 'region'] = 'Анкара'
df.loc[df['region'] == 'Черногория', 'region'] = 'Подгорица'
df.loc[df['region'] == 'Литва', 'region'] = 'Вильнюс'
df.loc[df['region'] == 'Венгрия', 'region'] = 'Будапешт'
df.loc[df['region'] == 'Румыния', 'region'] = 'Бухарест'
df.loc[df['region'] == 'Болгария', 'region'] = 'София'
df.loc[df['region'] == 'Канада', 'region'] = 'Оттава'
df.loc[df['region'] == 'Бангладеш', 'region'] = 'Дакка'
df.loc[df['region'] == 'Акташ (Узбекистан)', 'region'] = 'Акташ'
df.loc[df['region'] == 'эмират Дубай', 'region'] = 'Дубай'
df.loc[df['region'] == 'деревня Румянцево', 'region'] = 'Румянцево'
df.loc[df['region'] == 'Новоусманский район', 'region'] = 'Новая Усмань'
df.loc[df['region'] == 'Киров (Кировская область)', 'region'] = 'Киров'
df.loc[df['region'] == 'Петропавловка (Республика Бурятия)', 'region'] = 'Петропавловка'
df.loc[df['region'] == 'Иваново (Ивановская область)', 'region'] = 'Иваново'
df.loc[df['region'] == 'Moscow', 'region'] = 'Москва'
df.loc[df['region'] == 'Подольск (Московская область)', 'region'] = 'Подольск'
df.loc[df['region'] == 'посёлок Коммунарка', 'region'] = 'Коммунарка'
df.loc[df['region'] == 'г. Санкт-Петербург', 'region'] = 'Санкт-Петербург'
df.loc[df['region'] == 'посёлок Шушары', 'region'] = 'Шушары'
df.loc[df['region'] == 'Волжский (Волгоградская область)', 'region'] = 'Волжский'
df.loc[df['region'] == 'Penza', 'region'] = 'Пенза'
df.loc[df['region'] == 'г. Москва', 'region'] = 'Москва'
df.loc[df['region'] == 'Санкт-Петербург  м.пл.Ленина   Кондратьевский д 13', 'region'] = 'Санкт-Петербург'
df.loc[df['region'] == 'г.Москва', 'region'] = 'Москва'
df.loc[df['region'] == 'Рощино (Ленинградская область)', 'region'] = 'Рощино'
df.loc[df['region'] == 'г. Ставрополь ул. Апанасенковская 4', 'region'] = 'Ставрополь'
df.loc[df['region'] == 'деревня Покровское', 'region'] = 'Покровское'
df.loc[df['region'] == 'городской округ Клин', 'region'] = 'Клин'
df.loc[df['region'] == 'посёлок городского типа Малаховка', 'region'] = 'Малаховка'
df.loc[df['region'] == 'Московская', 'region'] = 'Москва'
df.loc[df['region'] == 'Автово', 'region'] = 'Санкт-Петербург'
df.loc[df['region'] == 'село Красное', 'region'] = 'Санкт-Петербург'

Далее применим функцию `city_to_country`:

<div class="alert alert-warning", style="border:solid coral 3px; padding: 20px">
<font size="4", color = "DimGrey"><b>⚠️ ÁCHTUNG!</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
        Дорогой ревьюер! 😊 
        Функция "city_to_country" имеет длительное время исполнения до 40 минут. Ознакомьтесь, пожалуйста, с результатами ее вычислений заранее в предпросмотре или загрузите готовую таблицу 'parsed_hh_preprocessed.csv' 👌

In [80]:
df['country'] = df.apply(city_to_country, axis=1)

In [81]:
# Проверим качество распределения по странам
df['country'].value_counts()

Россия         3661
Казахстан       212
Беларусь         98
Грузия           74
Узбекистан       49
Кипр             47
Армения:         35
Сербия           15
Азербайджан      12
Кыргызстан        9
Украина           9
Болгария          9
Польша            5
Юнайтед           5
Молдавия          4
Турция            3
Словения          2
состояния         2
Гора              2
Литва             2
Германия          2
Канада            1
Бангладеш         1
Латвия            1
мужчина           1
Румыния           1
Финляндия         1
Море              1
Венгрия           1
Name: country, dtype: int64

In [82]:
# Исправим недостатки, обнаруженные в классификации
df.loc[df['country']=='Армения:', 'country'] = 'Армения'
df.loc[df['country']=='Юнайтед', 'country'] = 'Объединенные Арабские Эмираты'
df.loc[df['country']=='состояния', 'country'] = 'Соединенные Штаты Америки'
df.loc[df['country']=='Гора', 'country'] = 'Черногория'
df.loc[df['country']=='мужчина', 'country'] = 'Вьетнам'
df.loc[df['country']=='Море', 'country'] = 'Грузия'
df.loc[df['region']=='Севастополь', 'country'] = 'Россия'

In [83]:
# Повторно проверяем качество классификации
df['country'].value_counts()

Россия                           3669
Казахстан                         212
Беларусь                           98
Грузия                             75
Узбекистан                         49
Кипр                               47
Армения                            35
Сербия                             15
Азербайджан                        12
Кыргызстан                          9
Болгария                            9
Польша                              5
Объединенные Арабские Эмираты       5
Молдавия                            4
Турция                              3
Литва                               2
Германия                            2
Соединенные Штаты Америки           2
Черногория                          2
Словения                            2
Венгрия                             1
Румыния                             1
Финляндия                           1
Вьетнам                             1
Канада                              1
Бангладеш                           1
Латвия      

Успешно проведено распределение данных по странам.

In [84]:
# Итоговый датасет
df.head()

,title,vacancy_type,work_experience,salary,currency,region,salary_min,salary_max,exp_min,exp_max,salary_mean,exp_mean,country
0,Python Разработчик,SE,1–3 года,от 200 000 до 1 500 000 KZT на руки,KZT,Астана,200000,1500000,1,3,850000,2,Казахстан
1,Full-stack Python Developer,SE,3–6 лет,от 4 500 до 5 000 EUR до вычета налогов,EUR,Астана,4500,5000,3,6,4750,5,Казахстан
2,Junior разработчик/программист в Python,SE,не требуется,от 150 000 до 200 000 KZT до вычета налогов,KZT,Нур-Султан (Астана),150000,200000,0,0,175000,0,Казахстан
3,Python Developer (Middle/Senior),SE,3–6 лет,от 2 000 до 4 000 USD на руки,USD,Алматы,2000,4000,3,6,3000,5,Казахстан
4,Backend разработчик,SE,1–3 года,з/п не указана,missing,Астана,0,0,1,3,0,2,Казахстан


**Значения столбцов**

* `title` - наименование вакансии;
* `vacancy_type` - буквенное обозначение категории вакансии;
* `work_experience` - требуемый опыт работы;
* `salary` - предложение работодателя по зарплате;
* `currency` - валюта;
* `region` - город в котором предлагается должность;
* `salary_min` - минимальное предложение работодателя по зарплате;
* `salary_max` - максимальное предложение работодателя по зарплате;
* `exp_min` - минимальный требуемый опыт работы;
* `exp_max` - максимальный требуемый опыт работы;
* `salary_mean` - среднее зарплатное предложение работодателя;
* `exp_mean` - средний требуемый опыт работы;
* `country` - страна, в которой предлагается должность.

In [85]:
# Запишем датасет в csv-таблицу
df.to_csv('parsed_hh_preprocessed.csv', index=False)

Произведем расчеты по региону.

In [86]:
# По регионам в рублях (средняя)
country_salary_rub_mean = (df[(df['salary_mean']!=0)&(df['currency']=='RUB')]
                         .groupby('country')[['salary_min', 'salary_max', 'salary_mean']]
                         .mean().round(2).sort_values('salary_mean', ascending=True))
country_salary_rub_mean

,salary_min,salary_max,salary_mean
country,,,
Азербайджан,80000.00,160000.00,120000.00
Кыргызстан,80000.00,170000.00,125000.00
Россия,102068.01,166962.86,134515.44
Армения,110000.00,190000.00,150000.00
Кипр,120000.00,250000.00,185000.00
Казахстан,193333.33,333333.33,263333.33
Соединенные Штаты Америки,325000.00,650000.00,487500.00


In [87]:
# По регионам во всех видах валюты (средняя)
country_salary_all_mean = (df[df['salary_mean']!=0]
                         .groupby(['country', 'currency'])[['salary_min', 'salary_max', 'salary_mean']]
                         .mean().round(2))
country_salary_all_mean

salary_min   salary_max  salary_mean
country                       currency                                       
Азербайджан                   RUB          80000.00    160000.00    120000.00
                              USD           2833.33      5000.00      3916.67
Армения                       EUR           3166.67      5000.00      4083.33
                              RUB         110000.00    190000.00    150000.00
                              USD           3000.00      5625.00      4312.50
Беларусь                      BYN            975.00      1950.00      1462.50
                              USD           2061.11      3388.89      2725.00
Болгария                      EUR           3431.25      5208.12      4319.75
Германия                      EUR           7500.00     10834.00      9167.00
Грузия                        EUR           2700.00      4800.00      3750.00
                              USD           2678.57      4678.57      3678.57
Казахстан                     EUR           4785.71      6142.86      5464.29
                              KZT         388471.43    711800.00    550135.71
                              RUB         193333.33    333333.33    263333.33
                              USD           2833.33      4466.67      3650.00
Канада                        USD           4000.00      8000.00      6000.00
Кипр                          EUR           2758.33      4905.56      3831.94
                              RUB         120000.00    250000.00    185000.00
Кыргызстан                    KGS          65000.00    130000.00     97500.00
                              RUB          80000.00    170000.00    125000.00
                              USD           1666.67      2666.67      2166.67
Латвия                        EUR           4500.00      5000.00      4750.00
Объединенные Арабские Эмираты USD           5000.00      7000.00      6000.00
Россия                        EUR           3966.67      5262.22      4614.44
                              RUB         102068.01    166962.86    134515.44
                              USD           3221.97      5290.91      4256.44
Соединенные Штаты Америки     RUB         325000.00    650000.00    487500.00
                              USD           4000.00      8000.00      6000.00
Узбекистан                    USD           1510.00      2680.00      2095.00
                              UZS       16166666.67  32333333.33  24250000.00

In [88]:
# По регионам в рублях (медианная)
country_salary_rub_median = (df[(df['salary_mean']!=0)&(df['currency']=='RUB')]
                         .groupby('country')[['salary_min', 'salary_max', 'salary_mean']]
                         .median().round(2).sort_values('salary_mean', ascending=True))
country_salary_rub_median

,salary_min,salary_max,salary_mean
country,,,
Азербайджан,80000.0,160000.0,120000.0
Кыргызстан,80000.0,170000.0,125000.0
Россия,100000.0,150000.0,125000.0
Армения,110000.0,190000.0,150000.0
Кипр,120000.0,250000.0,185000.0
Казахстан,200000.0,250000.0,235000.0
Соединенные Штаты Америки,325000.0,650000.0,487500.0


In [89]:
# По регионам во всех видах валюты (медианная)
country_salary_all_median = (df[df['salary_mean']!=0]
                         .groupby(['country', 'currency'])[['salary_min', 'salary_max', 'salary_mean']]
                         .median().round(2))
country_salary_all_median

salary_min  salary_max  salary_mean
country                       currency                                     
Азербайджан                   RUB          80000.0    160000.0     120000.0
                              USD           3000.0      6000.0       4500.0
Армения                       EUR           3000.0      5000.0       4500.0
                              RUB         110000.0    190000.0     150000.0
                              USD           3000.0      5750.0       4375.0
Беларусь                      BYN            975.0      1950.0       1462.5
                              USD           1250.0      2500.0       1875.0
Болгария                      EUR           3500.0      5432.5       4366.5
Германия                      EUR           7500.0     10834.0       9167.0
Грузия                        EUR           2500.0      5000.0       3750.0
                              USD           2500.0      4000.0       3000.0
Казахстан                     EUR           5000.0      7000.0       6000.0
                              KZT         350000.0    600000.0     450000.0
                              RUB         200000.0    250000.0     235000.0
                              USD           3000.0      5000.0       4000.0
Канада                        USD           4000.0      8000.0       6000.0
Кипр                          EUR           2500.0      5000.0       3750.0
                              RUB         120000.0    250000.0     185000.0
Кыргызстан                    KGS          65000.0    130000.0      97500.0
                              RUB          80000.0    170000.0     125000.0
                              USD           2000.0      3000.0       2750.0
Латвия                        EUR           4500.0      5000.0       4750.0
Объединенные Арабские Эмираты USD           5000.0      7000.0       6000.0
Россия                        EUR           4000.0      6000.0       5000.0
                              RUB         100000.0    150000.0     125000.0
                              USD           3000.0      5000.0       3875.0
Соединенные Штаты Америки     RUB         325000.0    650000.0     487500.0
                              USD           4000.0      8000.0       6000.0
Узбекистан                    USD           1500.0      2750.0       2250.0
                              UZS       18000000.0  36000000.0   27000000.0

#### 5.1.2. Расчет по группам вакансий

In [90]:
# По группам вакансий в рублях (средняя)
vtype_salary_rub_mean = (df[(df['salary_mean']!=0)&(df['currency']=='RUB')]
                         .groupby('vacancy_type')[['salary_min', 'salary_max', 'salary_mean']]
                         .mean().round(2).sort_values('salary_mean', ascending=True))
vtype_salary_rub_mean

,salary_min,salary_max,salary_mean
vacancy_type,,,
BD,25000.00,50000.00,37500.00
DL,40000.00,80000.00,60000.00
WB,74900.00,109840.00,92370.00
DE,80000.00,110000.00,95000.00
other,95962.15,157528.24,126745.21
DA,102241.38,156103.45,129172.41
SE,106780.80,175781.11,141280.96
DS,151329.67,262996.33,207163.00


In [91]:
# По группам вакансий во всех видах валюты (средняя)
vtype_salary_all_mean = (df[df['salary_mean']!=0]
                         .groupby(['vacancy_type', 'currency'])[['salary_min', 'salary_max', 'salary_mean']]
                         .mean().round(2))
vtype_salary_all_mean

salary_min   salary_max  salary_mean
vacancy_type currency                                       
BD           RUB          25000.00     50000.00     37500.00
DA           BYN           1250.00      2500.00      1875.00
             EUR           1500.00      3500.00      2500.00
             RUB         102241.38    156103.45    129172.41
             USD           3416.67      4833.33      4125.00
DE           RUB          80000.00    110000.00     95000.00
DL           RUB          40000.00     80000.00     60000.00
             USD           5000.00      6000.00      5500.00
DS           RUB         151329.67    262996.33    207163.00
             USD           2250.00      2750.00      2500.00
SE           BYN            700.00      1400.00      1050.00
             EUR           3782.73      5262.44      4522.58
             KGS          65000.00    130000.00     97500.00
             KZT         423958.33    820833.33    622395.83
             RUB         106780.80    175781.11    141280.96
             USD           2465.94      4101.45      3283.70
             UZS       11750000.00  23500000.00  17625000.00
WB           KZT         533333.33    666666.67    600000.00
             RUB          74900.00    109840.00     92370.00
             USD            750.00      1500.00      1125.00
other        EUR           3579.69      5364.53      4472.12
             KZT         227687.50    401625.00    314656.25
             RUB          95962.15    157528.24    126745.21
             USD           3446.94      5908.16      4677.55
             UZS       25000000.00  50000000.00  37500000.00

In [92]:
# По группам вакансий в рублях (медианная)
vtype_salary_rub_median = (df[(df['salary_mean']!=0)&(df['currency']=='RUB')]
                         .groupby('vacancy_type')[['salary_min', 'salary_max', 'salary_mean']]
                         .median().round(2).sort_values('salary_mean', ascending=True))
vtype_salary_rub_median

,salary_min,salary_max,salary_mean
vacancy_type,,,
BD,25000.0,50000.0,37500.0
DL,40000.0,80000.0,60000.0
WB,50000.0,90000.0,67500.0
DE,80000.0,110000.0,95000.0
other,80000.0,136382.0,110568.0
DA,100000.0,150000.0,130000.0
SE,100000.0,160000.0,130000.0
DS,150000.0,250000.0,200000.0


In [93]:
# По группам вакансий во всех видах валюты (медианная)
vtype_salary_all_median = (df[df['salary_mean']!=0]
                         .groupby(['currency', 'vacancy_type'])[['salary_min', 'salary_max', 'salary_mean']]
                         .median().round(2))
vtype_salary_all_median

salary_min  salary_max  salary_mean
currency vacancy_type                                     
BYN      DA                1250.0      2500.0       1875.0
         SE                 700.0      1400.0       1050.0
EUR      DA                1500.0      3500.0       2500.0
         SE                4000.0      5000.0       4750.0
         other             3750.0      5332.5       4500.0
KGS      SE               65000.0    130000.0      97500.0
KZT      SE              400000.0    775000.0     612500.0
         WB              400000.0    450000.0     400000.0
         other           215000.0    340000.0     277500.0
RUB      BD               25000.0     50000.0      37500.0
         DA              100000.0    150000.0     130000.0
         DE               80000.0    110000.0      95000.0
         DL               40000.0     80000.0      60000.0
         DS              150000.0    250000.0     200000.0
         SE              100000.0    160000.0     130000.0
         WB               50000.0     90000.0      67500.0
         other            80000.0    136382.0     110568.0
USD      DA                3000.0      4250.0       3875.0
         DL                5000.0      6000.0       5500.0
         DS                2250.0      2750.0       2500.0
         SE                2500.0      4000.0       3000.0
         WB                 750.0      1500.0       1125.0
         other             3000.0      5500.0       4250.0
UZS      SE            11750000.0  23500000.0   17625000.0
         other         25000000.0  50000000.0   37500000.0

### 5.2. Корреляция уровня опыта от зарплаты

In [94]:
df[df['salary_mean']!=0][['exp_mean', 'salary_mean']].corr()

,exp_mean,salary_mean
exp_mean,1.000000,0.026924
salary_mean,0.026924,1.000000


Значение корреляции крайне низкое и указывает на отсутствие связи между признаками.

### 5.3. Десять наиболее часто встречающихся должностей

In [95]:
print('Датасет содержит:', len(df['title'].unique()), 'уникальных должностей.')

Датасет содержит: 2909 уникальных должностей.


In [96]:
df['title'].value_counts().head(10)

Инженер-программист                                              38
Ведущий сетевой инженер в ЦОД                                    34
Python Developer                                                 30
Разработчик Python                                               30
Программист Python                                               27
Программист                                                      25
Специалист службы поддержки с техническими знаниями (Контест)    23
DevOps Engineer                                                  20
Python разработчик                                               18
Разработчик ETL                                                  16
Name: title, dtype: int64

In [97]:
df = pd.read_csv('parsed_hh_preprocessed.csv')

### 5.4. Процентное соотношение каждого региона по вакансиям от всех вакансий

In [98]:
p_dict = {}
for i in df['country'].unique():
    percent = (len(df[df['country']==i])*100)/len(df)
    p_dict[i] = round(percent, 2)
percent_inf = (pd.DataFrame(list(p_dict.items()), columns=['country', 'fraction'])
               .sort_values('fraction', ascending=False)
               .reset_index(drop=True))
percent_inf

,country,fraction
0,Россия,86.03
1,Казахстан,4.97
2,Беларусь,2.30
3,Грузия,1.76
4,Узбекистан,1.15
5,Кипр,1.10
6,Армения,0.82
7,Сербия,0.35
8,Азербайджан,0.28
9,Кыргызстан,0.21


### 5.5. Cамая высокооплачиваемая из групп вакансий, исходя из их средних зарплат

Для данных в рублях.

In [99]:
vtype_salary_rub_mean[vtype_salary_rub_mean['salary_mean']==vtype_salary_rub_mean['salary_mean'].max()]['salary_mean']

vacancy_type
DS    207163.0
Name: salary_mean, dtype: float64

Самыми высокооплачиваемыми вакансиями являются вакансии из категории 'Data Science'.

# Чек-лист

[x]  Сгруппируйте вакансии по направлениями (DS, DE, Software Engenering, etc.);  
[x]  Какая средняя и медианная зарплата по группам вакансий?  
[x]  Какая средняя и медианная зарплата по каждому региону?  
[x]  Какая самая высокооплачиваемая из групп вакансий, исходя из их средних зарплат?  
[x]  Какое процентное соотношение каждого региона по вакансиям от всех вакансий?  
[x]  Какая корреляция уровня опыта от зарплаты? (Преобразовать уровень опыта в числовой тип, если возможно. Например: ('MI': 0, 'SE': 1, 'EN': 2, 'EX': 3);  
[x]  Сколько должностей в наборе данных?  
[x]  Какие 10 наиболее часто встречающихся должностей?  
